In [315]:
import re
from nltk.stem import PorterStemmer
porter = PorterStemmer()
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
regex = re.compile('[^a-z ]')


In [316]:
def sanitizeBasic(sent):
    #remove all punctuations and numbers
    sent=regex.sub(' ', sent)
    sent=re.sub(r"[ ]+", " ", sent)
    return sent

In [317]:
def sanitizeStem(sent):
    #remove all punctuations, numbers and stem as well
    sent=regex.sub(' ', sent)
    sent=re.sub(r"[ ]+", " ", sent)
    sent= porter.stem(sent)
    return sent

In [318]:
def sanitizeStemStopWords(sent):
    #remove all punctuations, numbers and stop words. Do stemming 
    sent=regex.sub(' ', sent)
    sent=re.sub(r"[ ]+", " ", sent)
    sent=sent.split(" ")
    sent=' '.join([w for w in sent if w not in stop_words])
    sent= porter.stem(sent)
    
#     sent = re.sub(r"[\*\"“”\n\+\-\/\=\(\)‘•:\[\]\|’\!;\.\,\&\?]+", " ", sent)
#     sent = re.sub(r"[0-9]+", "", sent)
#     sent = re.sub(r"[\']+", "", sent)
#     sent=re.sub(r"[ ]+", " ", sent)
    
    return sent

In [319]:
sanitize=sanitizeStemStopWords

In [320]:
print(sanitize("It's hard to believe \"Memory of Trees\" came out 11 years ago;it has held up well over the passage of time.It's Enya's last great album before the New Age/pop of \"Amarantine\" and \"Day without rain.\" Back in 1995,Enya still had her creative spark,her own voice.I agree with the reviewer who said that this is her saddest album;it is melancholy,bittersweet,from the opening title song.\"Memory of Trees\" is elegaic&majestic.;\"Pax Deorum\" sounds like it is from a Requiem Mass,it is a dark threnody.Unlike the reviewer who said that this has a \"disconcerting\" blend of spirituality&sensuality;,I don't find it disconcerting at all.\"Anywhere is\" is a hopeful song,looking to possibilities.\"Hope has a place\" is about love,but it is up to the listener to decide if it is romantic,platonic,etc.I've always had a soft spot for this song.\"On my way home\" is a triumphant ending about return.This is truly a masterpiece of New Age music,a must for any Enya fan!".lower()))

hard believe memory trees came years ago held well passage time enya last great album new age pop amarantine day without rain back enya still creative spark voice agree reviewer said saddest album melancholy bittersweet opening title song memory trees elegaic majestic pax deorum sounds like requiem mass dark threnody unlike reviewer said disconcerting blend spirituality sensuality find disconcerting anywhere hopeful song looking possibilities hope place love listener decide romantic platonic etc always soft spot song way home triumphant ending return truly masterpiece new age music must enya fan 


In [321]:
print(sanitize('playing'))

play


In [322]:
import pandas as pd

In [323]:
data=pd.read_json('Music_reviews_json/reviews_Digital_Music_5.json/Music_Review_train.json', lines=True)

In [324]:
#contains the text data
dat=data[['reviewText']].copy()
# print(dat)

In [325]:
#contains the class data 
df_overall=data[['overall']].astype(int).copy()
# print(df_overall)

In [326]:
#find the frequencies of words in a sentence and returns a dictionary
def findFreq(sent):
    val=sent.split(" ")
    freq={}
    for s in val:
        if s not in freq:
            freq[s]=1
        else:
            freq[s]+=1
    return freq

In [327]:
dat['reviewText']=dat['reviewText'].str.lower()
# print(dat)

In [328]:
dat['reviewText']=dat['reviewText'].apply(sanitize)
# print(dat)

In [329]:
#find the length of each sentence in data
example_length=pd.DataFrame()
example_length['length']=dat['reviewText'].apply(len)
# print(example_length)

In [330]:
# print(dat.iloc[432]['reviewText'])

In [331]:
freq_df=pd.DataFrame()
freq_df['reviewText']=dat['reviewText'].apply(findFreq)
# print(freq_df)

In [332]:
vocab=set()
num_examples=df_overall.shape[0]
for d in range(num_examples):
    for k in freq_df.iloc[d]['reviewText'].keys():
        if k not in vocab:
            vocab.add(k)
# print(len(vocab))

In [333]:
vocab_size=len(vocab)

In [334]:
# print(vocab)

In [335]:
sum_denom={} #the summation of length of reviews depending upon their class
num_classes=5
for i in range(1,num_classes+1):
    sum_denom[i]=0
for i in range(num_examples):
    k=df_overall.iloc[i]['overall']
    sum_denom[k]+=example_length.iloc[i]['length']
# print(sum_denom)

In [336]:
sum_numer={} #the sum of the numerator for each theta
for word in vocab:
    sum_numer[word]={}
    for i in range(1,num_classes+1):
        sum_numer[word][i]=0
for i in range(num_examples):
    k=df_overall.iloc[i]['overall']
    d=freq_df.iloc[i]['reviewText']
    for j in d:
        sum_numer[j][k]+=d[j]


In [337]:
import math
thetas={}
for word in vocab:
    thetas[word]={}
    for i in range(1,num_classes+1):
        thetas[word][i]=math.log((sum_numer[word][i]+1)/(sum_denom[i]+vocab_size+1)) # +1 in denominator for unk token(words not in vobabulary)

In [338]:
# print(thetas['have'])
# print(thetas['counterparts'])

In [350]:
#calculate p(y=k)
df_overall_size=df_overall.groupby('overall')
df_overall_size=df_overall_size['overall'].agg(['size'])
df_overall_size['size']=df_overall_size['size']/num_examples
df_overall_size['size']=df_overall_size['size'].apply(math.log)
# print(df['size'])

          size
overall       
1         2529
2         2638
3         5634
4        13267
5        25932


In [340]:
#unk tokens
unk_p={i:math.log(1/(sum_denom[i]+vocab_size+1)) for i in range(1,num_classes+1)}

In [341]:
#training done. testing
test=pd.read_json('Music_reviews_json/reviews_Digital_Music_5.json/Music_Review_test.json', lines=True)

In [342]:
testR=pd.DataFrame()
testR['reviewText']=test['reviewText'].copy().str.lower().apply(sanitize)
testO=pd.DataFrame()
testO['overall']=test['overall'].astype(int).copy()

In [343]:
def uniqueWords(sent):
    arr=sent.split(" ")
    s=set()
    for a in arr:
        if a not in s:
            s.add(a)
    return s

In [344]:
testR['reviewText']=testR['reviewText'].apply(uniqueWords)

In [345]:
# print(testR)

In [346]:
def predictV(setS):
    d={i:df_overall_size['size'][i] for i in range(1, num_classes+1)}
    for s in setS:
        if s not in thetas:
            for i in range(1, num_classes+1):
                d[i]+=unk_p[i]
        else:
            for i in range(1, num_classes+1):
                d[i]+=thetas[s][i]
    return max(d,key=d.get)

In [347]:
predict=pd.DataFrame()
predict['val']=testR['reviewText'].apply(predictV)

In [354]:
# print(predict)

       val
0        5
1        2
2        3
3        5
4        3
...    ...
13995    5
13996    4
13997    4
13998    4
13999    4

[14000 rows x 1 columns]


In [349]:
a=0
for i in range(14000):
    if predict.iloc[i]['val']==testO.iloc[i]['overall']:
        a+=1;
print(a/14000)

0.5249285714285714


In [352]:
#confusion matrix
from sklearn import metrics
metrics.confusion_matrix(testO, predict)

array([[ 130,   46,   17,    9,   26],
       [  71,  116,   70,   25,   44],
       [  72,  267,  304,  214,  229],
       [ 240,  431,  529,  665, 1243],
       [1008,  827,  432,  851, 6134]])

In [355]:
print(testO.groupby('overall').agg('size'))

overall
1     228
2     326
3    1086
4    3108
5    9252
dtype: int64
